In [7]:
import glob
import os
import pandas as pd

In [1]:
BASE_FOLDER = f"/home/app/Desktop/Frames_Analysis"
VIDEO_PATH = f"{BASE_FOLDER}/game_videos"
RECORDED_FRAMES_PATH = f"{BASE_FOLDER}/game_data/recorded_frames"
LABEL = "pilot2"


In [ ]:
#KCC DESKTOP
BASE_FOLDER = f"/Users/katecandon/Desktop/ExpZ"
VIDEO_PATH = f"{BASE_FOLDER}/game_videos"
RECORDED_FRAMES_PATH = f"{BASE_FOLDER}/game_data/recorded_frames"
LABEL = "EXPZ_check"

In [5]:
def create_csv_folder_info(base_folder, recorded_frames_path, video_path,label):
    if base_folder[0]==".":
        recorded_frames_path = f".{recorded_frames_path}"
        base_folder = f".{base_folder}"

    folders = !ls {recorded_frames_path}
    
    folder_info = []
    for folder in folders:
        check, webcam, wstart, wend, gamescreen = 0,0,0,0,0
        pid = folder[:folder.find("_")]
        if folder.find("vo") == -1:
            folder_type = "CHECK"
            check = !ls {recorded_frames_path}/{folder}/check | wc -l
            check = int(check[0])
        else:
            folder_type = "GAME"
            try:
                webcam = !ls {recorded_frames_path}/{folder}/webcam | wc -l
                webcam = int(webcam[0])
            except:
                webcam = 0
            try:
                wstart = !ls {recorded_frames_path}/{folder}/webcam_start | wc -l
                wstart = int(wstart[0])
            except:
                wstart = 0
            try:
                wend = !ls {recorded_frames_path}/{folder}/webcam_end | wc -l
                wend = int(wend[0])
            except:
                wend = 0
            try:
                gamescreen = !ls {recorded_frames_path}/{folder}/gamescreen | wc -l
                gamescreen = int(gamescreen[0])
            except:
                gamescreen = 0
        folder_info.append([folder, pid, folder_type, check, wstart, webcam, gamescreen, wend])
    folder_df = pd.DataFrame(data = folder_info, columns=["Folder","PID","Type","Check","Webcam_start","Webcam","Gamescreen","Webcam_end"])
    folder_df.to_csv(f"{base_folder}/{label}_folderinfo.csv")
    return folders, folder_df

In [8]:
folders, folder_df = create_csv_folder_info(BASE_FOLDER, RECORDED_FRAMES_PATH, VIDEO_PATH, LABEL)

In [9]:
def create_videos(video_path, rf_path, label, folders, folder_df):
    i = 1
    for folder in folders:
        print(f"Working on folder {i} of {len(folders)}")
        print(folder_df[folder_df['Folder']==folder].Type.item())
        if folder_df[folder_df['Folder']==folder].Type.item() == "CHECK":
            webcam_only(video_path,rf_path,folder,"check")
        else:
            try:
                webcam_only(video_path,rf_path,folder,"webcam_start")
            except:
                print("START ERROR: ", folder)
            try:
                webcam_only(video_path,rf_path,folder,"webcam_end")
            except:
                print("END ERROR: ", folder)
            try:
                game_and_webcam(video_path,rf_path,folder)
            except:
                print("GAME ERROR: ", folder)
        i += 1
            
        

In [10]:
def webcam_only(video_path, rf_path,folder,subfolder):
    path = f"{rf_path}/{folder}/{subfolder}/*.jpg"
    images = glob.glob(path)
    
    values = {}
    for im in images:
        im = im.split("/")[-1]
        try:
            [w,num,millis] = im.split("_")
            millis = int(millis[1:-4])/1000
        except:
            [w,num,millis,cm] = im.split("_")
            millis = int(millis[1:])/1000
        num = int(num)
        values[millis] = {"file":im,"frame":num}
    sorted_ims = sorted(values.keys())
    print(path)
    min_millis = min(sorted_ims)
    
    if not os.path.exists(os.path.dirname(f"{video_path}/{folder}/")):
        os.makedirs(os.path.dirname(f"{video_path}/{folder}/"))
        
        
        
    with open(f"{video_path}/{folder}/{subfolder}.cmd","+w") as f:
        for i in range(len(sorted_ims)):
            this_millis = sorted_ims[i]
            time = this_millis - min_millis
            f.write(f"{time} drawtext reinit 'text={values[this_millis]['frame']}'; \n")
    
    with open(f"{video_path}/{folder}/{subfolder}.ffconcat","+w") as f:
        f.write("ffconcat version 1.0 \n")
        for i in range(len(sorted_ims)-1):
            this_millis = sorted_ims[i]
            next_millis = sorted_ims[i+1]
            f.write(f"file {rf_path}/{folder}/{subfolder}/{values[this_millis]['file']} \n")
            f.write(f"duration {next_millis-this_millis} \n")
        f.write(f"file {rf_path}/{folder}/{subfolder}/{values[next_millis]['file']} \n")    
    ! ffmpeg -safe 0 -i {video_path}/{folder}/{subfolder}.ffconcat -vf "scale=trunc(iw/2)*2:trunc(ih/2)*2,sendcmd=f={video_path}/{folder}/{subfolder}.cmd,drawtext=text='':x=10:y=345:fontcolor=white:fontsize=12" -pix_fmt yuv420p -y {video_path}/{folder}/{subfolder}{folder}.mp4 >/dev/null 2>&1

In [11]:
def game_and_webcam(video_path,rf_path,folder):
    w_path = f"{rf_path}/{folder}/webcam/*.jpg"
    g_path = f"{rf_path}/{folder}/gamescreen/*.jpg"
    w_images = glob.glob(w_path)
    g_images = glob.glob(g_path)
    
    w_values = {}
    
    g_values = {}
    
    for im in w_images:
        im = im.split("/")[-1]
        try:
            [w,num,millis] = im.split("_")
            millis = int(millis[1:-4])/1000
        except:
            [w,num,millis,cm] = im.split("_")
            millis = int(millis[1:])/1000
        num = int(num)
        w_values[millis] = {"file":im,"frame":num}
    
    for im in g_images:
        im = im.split("/")[-1]
        try:
            [w,num,millis] = im.split("_")
            millis = int(millis[1:-4])/1000
        except:
            [w,num,millis,cm] = im.split("_")
            millis = int(millis[1:])/1000
        num = int(num)
        g_values[millis] = {"file":im,"frame":num}
        
    sorted_w = sorted(w_values.keys())
    sorted_g = sorted(g_values.keys())
    
    min_millis = min(min(sorted_w),min(sorted_g))
    offset_w = min(sorted_w) - min_millis
    offset_g = min(sorted_g) - min_millis
    
    if not os.path.exists(os.path.dirname(f"{video_path}/{folder}/")):
        os.makedirs(os.path.dirname(f"{video_path}/{folder}/"))
    
    print("starting gamescreen")
    with open(f"{video_path}/{folder}/gamescreen_frames.cmd","+w") as f:
        for i in range(len(sorted_g)):
            this_millis = sorted_g[i]
            time = this_millis - min_millis
            f.write(f"{time} drawtext reinit 'text={g_values[this_millis]['frame']}'; \n")
    
    with open(f"{video_path}/{folder}/gamescreen.ffconcat","+w") as f:
        f.write("ffconcat version 1.0 \n")
        if offset_g > 0:
            this_millis = sorted_g[0]
            f.write(f"file {rf_path}/{folder}/gamescreen/{g_values[this_millis]['file']} \n")
            f.write(f"duration {offset_g} \n")
        for i in range(len(sorted_g)-1):
            this_millis = sorted_g[i]
            next_millis = sorted_g[i+1]
            f.write(f"file {rf_path}/{folder}/gamescreen/{g_values[this_millis]['file']} \n")
            f.write(f"duration {next_millis-this_millis} \n")
        f.write(f"file {rf_path}/{folder}/gamescreen/{g_values[next_millis]['file']} \n")
    print(f"{video_path}/{folder}/gamescreen.ffconcat")
    ! ffmpeg -safe 0 -i {video_path}/{folder}/gamescreen.ffconcat -vf "scale=trunc(iw/2)*2:trunc(ih/2)*2,sendcmd=f={video_path}/{folder}/gamescreen_frames.cmd,drawtext=text='':x=10:y=580:fontcolor=white:fontsize=20" -pix_fmt yuv420p -y {video_path}/{folder}/gamescreen_{folder}.mp4 >/dev/null 2>&1

    print("starting webcam")
    with open(f"{video_path}/{folder}/webcam_frames.cmd","+w") as f:
        for i in range(len(sorted_w)):
            this_millis = sorted_w[i]
            time = this_millis - min_millis
            f.write(f"{time} drawtext reinit 'text={w_values[this_millis]['frame']}'; \n")   
    
    with open(f"{video_path}/{folder}/webcam.ffconcat","+w") as f:
        f.write("ffconcat version 1.0 \n")
        if offset_w > 0:
            this_millis = sorted_w[0]
            f.write(f"file {rf_path}/{folder}/webcam/{w_values[this_millis]['file']} \n")
            f.write(f"duration {offset_w} \n")
        for i in range(len(sorted_w)-1):
            this_millis = sorted_w[i]
            next_millis = sorted_w[i+1]
            f.write(f"file {rf_path}/{folder}/webcam/{w_values[this_millis]['file']} \n")
            f.write(f"duration {next_millis-this_millis} \n")
        f.write(f"file {rf_path}/{folder}/webcam/{w_values[next_millis]['file']} \n")    
    ! ffmpeg -safe 0 -i {video_path}/{folder}/webcam.ffconcat -vf "scale=trunc(iw/2)*2:trunc(ih/2)*2,sendcmd=f={video_path}/{folder}/webcam_frames.cmd,drawtext=text='':x=10:y=345:fontcolor=white:fontsize=12" -pix_fmt yuv420p -y {video_path}/{folder}/webcam_{folder}.mp4 >/dev/null 2>&1

    
    print("starting join")
    ! ffmpeg -i {video_path}/{folder}/gamescreen_{folder}.mp4 -i {video_path}/{folder}/webcam_{folder}.mp4 -filter_complex "[1:v][0:v]scale2ref=oh*mdar:h=in_h:[v1][v0];[v0][v1]hstack[vo]" -map "[vo]" -pix_fmt yuv420p -y {video_path}/{folder}/join_{folder}.mp4 >/dev/null 2>&1

In [ ]:
game_and_webcam(VIDEO_PATH,RECORDED_FRAMES_PATH,"P570b7b5fb9b52b00102d96f9_von_m1_g1_t2021_6_22_13_34")

In [12]:
create_videos(VIDEO_PATH, RECORDED_FRAMES_PATH, LABEL, folders, folder_df)

Working on folder 1 of 1
GAME
/home/app/Desktop/Frames_Analysis/game_data/recorded_frames/Ptest_voff_m1_g0_t2023_7_2_17_36/webcam_start/*.jpg
/home/app/Desktop/Frames_Analysis/game_data/recorded_frames/Ptest_voff_m1_g0_t2023_7_2_17_36/webcam_end/*.jpg
starting gamescreen
/home/app/Desktop/Frames_Analysis/game_videos/Ptest_voff_m1_g0_t2023_7_2_17_36/gamescreen.ffconcat
starting webcam
starting join


In [ ]:
#webcam_only(BASE_PATH,RECORDED_FRAMES_PATH,"P5f14e0f456500058ee865863_t2021_6_11_12_14","check")